In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
#подготовка данных
data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep='\t' )

In [3]:
#1. Вывести: кол-во наблюдений в датасете
size = data.shape[0]
print(size)

4622


In [4]:
#2. Вывести названия столбцов
names = data.columns
print(names)

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')


In [5]:
#3. Определить самую частую позицию (item) в заказе
data['item_name'].value_counts().idxmax()

'Chicken Bowl'

In [ ]:
#4. Построить гистрограмму частоты заказов по позициям (item)

In [10]:
#5. Измените тип переменной item_price с помощью лямбды функции
data['item_price'] = data.apply(lambda x: float(x['item_price'].replace('$', '')),axis=1)
data.head(5)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


In [ ]:
#6. Построить гистограмму кол-во денег заработанных по каждой позиции (item)

In [11]:
#7. Средняя сумма заказа? (минимум 2 способа)
# первый способ
data['item_price'].sum() / data['order_id'].max()


18.811428571428575

In [ ]:
#8. Выведите среднее, минимальное и максимальное, медианное значения позиций в заказе

In [ ]:
#9. Определить статистику заказов стейков, а также статистику заказов прожарки

In [ ]:
#10. Добавить новый столбец цен на каждую позицию в заказе в рублях

In [ ]:
#11. Сгруппировать заказы по входящим позициям в него. Отдельно сгруппировать по стейкам во всех видах прожарках

In [ ]:
#12. Определить цену по каждой позиции в отдельности